## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ilhabela,BR,-23.7781,-45.3581,64.51,81,0,4.41,clear sky
1,1,Pacific Grove,US,36.6177,-121.9166,51.42,83,15,4.00,few clouds
2,2,Butaritari,KI,3.0707,172.7902,82.49,76,100,8.81,overcast clouds
3,3,Tasiilaq,GL,65.6145,-37.6368,24.96,56,100,6.80,overcast clouds
4,4,Itacare,BR,-14.2775,-38.9967,75.45,89,98,5.68,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ilhabela,BR,-23.7781,-45.3581,64.51,81,0,4.41,clear sky
2,2,Butaritari,KI,3.0707,172.7902,82.49,76,100,8.81,overcast clouds
4,4,Itacare,BR,-14.2775,-38.9967,75.45,89,98,5.68,overcast clouds
5,5,Kavieng,PG,-2.5744,150.7967,84.07,79,92,14.05,moderate rain
6,6,Atuona,PF,-9.8000,-139.0333,78.55,68,77,13.51,broken clouds
7,7,Belmonte,BR,-15.8631,-38.8828,72.10,92,100,5.03,moderate rain
8,8,Richards Bay,ZA,-28.7830,32.0377,73.09,91,52,3.42,broken clouds
11,11,Mataura,NZ,-46.1927,168.8643,60.48,40,59,8.95,broken clouds
13,13,Puerto Ayora,EC,-0.7393,-90.3518,70.02,88,97,6.20,overcast clouds
14,14,Rikitea,PF,-23.1203,-134.9692,75.67,75,78,18.79,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
import numpy as np

clean_df=preferred_cities_df.replace('', np.nan)
nan_values=clean_df[clean_df.isna().any(axis=1)]
nan_values

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
79,79,Opuwo,NaN,-18.0607,13.8400,71.22,71,75,12.91,broken clouds
206,206,Luderitz,NaN,-26.6481,15.1594,60.13,79,0,5.64,clear sky
520,520,Rehoboth,NaN,-23.3167,17.0833,70.32,48,100,9.08,overcast clouds
566,566,Eenhana,NaN,-17.4667,16.3333,67.35,83,100,1.74,overcast clouds
678,678,Okahandja,NaN,-21.9833,16.9167,71.91,62,100,6.38,overcast clouds


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df.dropna(subset=['Country'], inplace=True)


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ilhabela,BR,64.51,clear sky,-23.7781,-45.3581,
2,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,
4,Itacare,BR,75.45,overcast clouds,-14.2775,-38.9967,
5,Kavieng,PG,84.07,moderate rain,-2.5744,150.7967,
6,Atuona,PF,78.55,broken clouds,-9.8000,-139.0333,
7,Belmonte,BR,72.10,moderate rain,-15.8631,-38.8828,
8,Richards Bay,ZA,73.09,broken clouds,-28.7830,32.0377,
11,Mataura,NZ,60.48,broken clouds,-46.1927,168.8643,
13,Puerto Ayora,EC,70.02,overcast clouds,-0.7393,-90.3518,
14,Rikitea,PF,75.67,broken clouds,-23.1203,-134.9692,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace("", np.nan)
clean_hotel_df.dropna(subset = ["Hotel Name"], inplace=True)

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Ilhabela,BR,64.51,clear sky,-23.7781,-45.3581,Ilha Flat Hotel
2,Butaritari,KI,82.49,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
4,Itacare,BR,75.45,overcast clouds,-14.2775,-38.9967,Ecoporan Hotel Charme Spa & Eventos
5,Kavieng,PG,84.07,moderate rain,-2.5744,150.7967,Nusa Island Retreat
6,Atuona,PF,78.55,broken clouds,-9.8000,-139.0333,Villa Enata


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"] 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))